<a href="https://colab.research.google.com/github/Pabloacostatec/iayaa-Equipo-25/blob/main/MNA_IAyAA_semana_10_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

*   
*   
*   
*   
*   


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [ ]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.

import requests, zipfile
from io import BytesIO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import power_transform
from sklearn.impute import SimpleImputer
from sklearn.feature_selection._base import SelectorMixin
from sklearn.feature_extraction.text import _VectorizerMixin
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt 
import seaborn as sns
import math as math

In [ ]:
#This method Imports zip directly
# importing necessary modules

print('Downloading started')

#Defining the zip file URL
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00232/RCdata.zip'

# Split URL to get the file name
filename = url.split('/')[-1]

# Downloading the file by sending the request to the URL
#Para poder usar request necesitamos la libreria import requests, zipfile
from io import BytesIO

req = requests.get(url)
print('Downloading Completed')

# extracting the zip file contents
zipfile= zipfile.ZipFile(BytesIO(req.content))
zipfile.extractall('/content/sample_data/Restaurant_consumer')

Ratingdf = pd.read_csv('/content/sample_data/Restaurant_consumer/rating_final.csv',sep=',', encoding_errors='ignore')
Geodf = pd.read_csv('/content/sample_data/Restaurant_consumer/geoplaces2.csv',sep=',', encoding_errors='ignore')

In [ ]:
Ratingdf

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2
...,...,...,...,...,...
1156,U1043,132630,1,1,1
1157,U1011,132715,1,1,0
1158,U1068,132733,1,1,0
1159,U1068,132594,1,1,1


In [ ]:
Geodf

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincn de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,132866,22.141220,-100.931311,0101000020957F000013871838EC4A58C1B5DF74F8E396...,Chaires,Ricardo B. Anaya,San Luis Potosi,San Luis Potosi,Mexico,?,...,No_Alcohol_Served,not permitted,informal,completely,medium,?,familiar,f,closed,none
126,135072,22.149192,-101.002936,0101000020957F0000E7B79B1DB94758C1D29BC363D8AA...,Sushi Itto,Venustiano Carranza 1809 C Polanco,San Luis Potosi,SLP,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,sushi-itto.com.mx,familiar,f,closed,none
127,135109,18.921785,-99.235350,0101000020957F0000A6BF695F136F5AC1DADF87B20556...,Paniroles,?,?,?,?,?,...,Wine-Beer,not permitted,informal,no_accessibility,medium,?,quiet,f,closed,Internet
128,135019,18.875011,-99.159422,0101000020957F0000B49B2E5C6E785AC12F9D58435241...,Restaurant Bar Coty y Pablo,Paseo de Las Fuentes 24 Pedregal de Las Fuentes,Jiutepec,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,closed,none


#**Ejercicio-1.** 

Descarga los archivos y genera la matriz de utilidad en la cual los renglones son los usuarios 
(user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluaciones 
generales dadas por los usuarios (rating). En particular indica el tamaño de dicha matriz y despliega 
sus primeros 10 renglones. 

In [ ]:
#Se substitutye placeID por name
key_to_replace = list(Ratingdf['placeID'])
look_up_dictionary = dict(zip(Geodf['placeID'], Geodf['name']))
Ratingdf['name'] = [look_up_dictionary[item] for item in key_to_replace]

In [ ]:
Printdf = Ratingdf.drop(['placeID','food_rating','service_rating'], axis = 1)
Printdf = Printdf.set_index(['userID', 'name'])['rating'].unstack()
Printdf.replace(np.nan, 0, inplace = True)
print('El tamaño de la matriz es: ', Printdf.size)
Printdf.head(10)

El tamaño de la matriz es:  17802


name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El Rincón de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1003,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1004,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
U1005,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1006,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1007,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1008,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1009,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#**Ejercicio-2.**

Con el objetivo de obtener la información con respecto a los vectores latentes de los restaurantes, 
obtener dicha factorización SVD utilizando todas las componentes posibles, es decir, usando todos 
sus valores singulares. En particular indica la cantidad de valores singulares obtenidos. 

In [ ]:
def Apply_SVD_all_components(Summary_df, score_column, Interest_column):

  #name it easier
  df = Summary_df

  #see what is the column of interest for SVD
  AllColumns = df.columns.values
  Discarded = [score_column,Interest_column]
  Extract_index = [elem for elem in AllColumns if elem not in Discarded]
  Index_column = Extract_index[0]

  #print(Index_column)

  #set index and unstack column of interest
  df = df.set_index([Index_column, Interest_column])[score_column].unstack()
  df.replace(np.nan, 0, inplace = True)

  #print(df)

  #Since the unstacked is the column of interest, transpose
  X = df.T

  #print(X)

  #Calulate number of components for SVD

  num_components = Summary_df[Index_column].drop_duplicates().shape[0]

  #print(num_components)

  SVD = TruncatedSVD(n_components= num_components-1, random_state=1)  
  fittedSVD = SVD.fit_transform(X)

  #print((SVD.singular_values_))

  return fittedSVD, X

In [ ]:
def Apply_SVD_select_components(Summary_df, score_column, Interest_column,Num_components):

  #name it easier
  df = Summary_df

  #see what is the column of interest for SVD
  AllColumns = df.columns.values
  Discarded = [score_column,Interest_column]
  Extract_index = [elem for elem in AllColumns if elem not in Discarded]
  Index_column = Extract_index[0]

  #print(Index_column)

  #set index and unstack column of interest
  df = df.set_index([Index_column, Interest_column])[score_column].unstack()
  df.replace(np.nan, 0, inplace = True)

  #print(df)

  #Since the unstacked is the column of interest, transpose
  X = df.T

  #print(X)

  #print(num_components)

  SVD = TruncatedSVD(n_components= Num_components, random_state=1)  
  fittedSVD = SVD.fit_transform(X)

  #print((SVD.singular_values_))

  return fittedSVD, X

In [ ]:
#Se aplica SVD para todos los componentes

df_for_SVD = Ratingdf.drop(['placeID','food_rating','service_rating'], axis = 1)
SVDresult, tMatrix = Apply_SVD_all_components(df_for_SVD, 'rating', 'name')
print('Se tienen: ',len(np.unique(SVDresult)), ' valores singulares')
print('la matriz de valores singulares es: \n', SVDresult)

Se tienen:  16641  valores singulares
la matriz de valores singulares es: 
 [[ 2.23152914e-01 -3.35148397e-01  9.25716874e-01 ...  7.94749489e-03
   4.30362805e-16  2.34242182e-30]
 [-1.02397730e-17 -1.20313879e-16  1.93013320e-16 ...  8.59202396e-16
  -1.68889636e-17  8.79225538e-30]
 [ 3.04948374e+00  2.56481245e+00 -4.80893676e-01 ... -2.01501322e-03
  -4.01732985e-16  3.53373719e-30]
 ...
 [ 8.68871858e-01  1.45545704e+00  1.81260610e+00 ...  3.94017886e-03
  -5.38905613e-17  5.02231013e-30]
 [-9.34815887e-17 -2.65225890e-17 -3.30719621e-16 ... -5.04180850e-17
  -9.05529866e-18 -1.10152800e-29]
 [-7.39924403e-17 -4.90162646e-17 -2.69947956e-16 ...  1.14536504e-15
  -1.89196457e-17  1.76991036e-29]]


#**Ejercicio-3.**

Usando la métrica de correlación de Pearon, obtener las 10 mejores recomendaciones de este 
modelo no supervisado con base a la información de alguien que evaluó bien de manera general al 
“Subway”. Despliega también el valor numérico de la correlación de Pearson de cada una de estas 10 
recomendaciones. NOTA: No debe estar incluido “Subway” dentro de las recomendaciones.

In [ ]:
#Se aplica SVD para un número de componentes específico, en este caso 10

df_for_SVD = Ratingdf.drop(['placeID','food_rating','service_rating'], axis = 1)
SVDresult, tMatrix = Apply_SVD_select_components(df_for_SVD, 'rating', 'name',10)
print('la matriz de valores singulares es: \n', SVDresult)
print( 'la matriz tiene la siguiente forma: ',SVDresult.shape)

la matriz de valores singulares es: 
 [[ 2.23151748e-01 -3.35292160e-01  9.24898831e-01 ... -4.36209184e-01
   1.38120097e-01  5.44332273e-02]
 [ 3.73103113e-07 -3.47361366e-05  1.85667435e-04 ...  2.31099673e-03
  -4.21402188e-03  1.02892590e-02]
 [ 3.04948482e+00  2.56501400e+00 -4.79142255e-01 ...  8.81153631e-01
   5.45899118e-01 -1.58109887e+00]
 ...
 [ 8.68874314e-01  1.45586940e+00  1.81398999e+00 ...  8.44858221e-01
  -1.03375255e+00 -4.34266384e-01]
 [ 1.85741208e-07  2.87026537e-05 -1.41531000e-04 ...  9.67088593e-04
   1.85188285e-03  3.01278744e-03]
 [ 7.05914483e-07  1.63804231e-04  1.08846127e-03 ...  7.24073912e-03
   9.50462602e-03  5.23989250e-03]]
la matriz tiene la siguiente forma:  (129, 10)


In [ ]:
corrMx = np.corrcoef(SVDresult, )

me_gusta = "Subway"

names = Printdf.columns
names_list = list(names)
id_megusta = names_list.index(me_gusta)

id_megusta

corr_recomienda = corrMx[id_megusta]
#print(corr_recomienda.shape)
#print(corr_recomienda)

corr_recomienda[id_megusta]

list(names[(corr_recomienda > .009) & (corr_recomienda < 0.99)])
#list(corr_recomienda)

['El Rincn de San Francisco',
 'La Cantina Restaurante',
 'La Posada del Virrey',
 'Luna Cafe',
 'Restaurant de Mariscos de Picon',
 'Restaurante Bar El Gallinero',
 'Restaurante Marisco Sam',
 'Restaurante Versalles',
 'Restaurante de Mariscos la Langosta',
 'Restaurante la Gran Via',
 'Unicols Pizza',
 'cafe punta del cielo',
 'el pueblito',
 'la parroquia',
 'rockabilly ']

#**Ejercicio-4.**

In [ ]:
None

#**Ejercicio-5.**

In [ ]:
None

#**Ejercicio-6.**

In [ ]:
None

#**Ejercicio-7.**

In [ ]:
None

#**Ejercicio-8.**

In [ ]:
None

###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**